In [1]:
from sktime.transformers.compose import ColumnConcatenator
from sktime.classifiers.compose import TimeSeriesForestClassifier
from sktime.classifiers.dictionary_based.boss import BOSSEnsemble
from sktime.classifiers.compose import ColumnEnsembleClassifier
from sktime.classifiers.shapelet_based import ShapeletTransformClassifier
from sktime.datasets import load_basic_motions
from sktime.pipeline import Pipeline

import numpy as np
import pandas as pd


#cleaning up whatever happened in loading data 
#feed in the data 
from sktime.utils.load_data import from_long_to_nested
import pandas as pd 
import numpy as np 


def reformatData(target, file_name):
    #extracting the data from the csv as a dataframe and reformatting it 
    raw_df= pd.read_csv(file_name)

    long_table_df= raw_df.melt(id_vars=["event", "name","start time", "end time","channel"], 
            var_name="anindex", 
            value_name="value")

    sorted_long_table_df=long_table_df.sort_values(by=['event','name','start time','channel'], axis=0)

    unique_dim_ids = sorted_long_table_df.iloc[:, 4].unique()

    for i in range(len(unique_dim_ids)):
        my_channel=unique_dim_ids[i]
        sorted_long_table_df['channel']=sorted_long_table_df['channel'].replace({my_channel:i})
    unique_start_time = sorted_long_table_df.iloc[:, 2].unique()

    for i in range(len(unique_start_time)):
        my_time=unique_start_time[i]
        sorted_long_table_df['start time']=sorted_long_table_df['start time'].replace({my_time:i})

    
    sorted_long_table_df_stripped=sorted_long_table_df.drop(columns=['event','name','end time'])

    sorted_long_table_df_stripped.head()
    df_nested = from_long_to_nested(sorted_long_table_df_stripped)


    target='event'
    new_unique_start_time=sorted_long_table_df.iloc[:, 2].unique()
    labels=[]
    for e in new_unique_start_time:
        x=sorted_long_table_df.loc[sorted_long_table_df['start time']==e,[target]].iloc[0][0]
        labels.append(x)

    np_labels= np.asarray(labels, dtype=np.str)
    
    return df_nested, np_labels 

def splitTestTrain(X, y, percent_train):
    msk = np.random.rand(len(X)) < percent_train
    ytrain=y[msk]
    ytest=y[~msk]
    Xtrain=X[msk]
    Xtest=X[~msk]
    
    return Xtrain, ytrain, Xtest, ytest

def concatenateMethod(Classifier, x_train, y_train, x_test, y_test, estimators):
    steps = [
    ('concatenate', ColumnConcatenator()),
    ('classify', Classifier(n_estimators=estimators))]
    clf = Pipeline(steps)
    clf.fit(x_train, y_train)
    return clf.score(x_test, y_test)

In [2]:
target='event'
file_name="../scripts/data/2013e.csv"
unique_dimension_per_case='starting time'
X, y=reformatData(target,file_name)
percent_train=0.8
Xtrain, ytrain, Xtest, ytest= splitTestTrain(X, y, percent_train)

should_profile=True 

In [4]:
model_acc= concatenateMethod(TimeSeriesForestClassifier,Xtrain, ytrain, Xtest, ytest, estimators=10)

In [5]:
print (model_acc)

0.5714285714285714
